In [1]:
import torchvision
model = torchvision.models.quantization.mobilenet_v2(pretrained=True, quantize=True)

/home/luca/.local/lib/python3.8/site-packages/torch/quantization/observer.py:122: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/luca/.local/lib/python3.8/site-packages/torch/quantization/observer.py:244: UserWarning: must run observer before calling calculate_qparams.                                        Returning default scale and zero point 
  warnings.warn(
[W TensorImpl.h:1156] Warning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (function operator())


In [2]:
from PIL import Image
from torchvision import transforms
import torch

input_image = Image.open('../data/hymenoptera_data/train/ants/0013035.jpg')
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
#print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
#print(probabilities)

# Download ImageNet labels
#!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("../data/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

ant 0.9550977349281311
cockroach 0.009668114595115185
bee 0.008295673877000809
scorpion 0.0044966693967580795
dragonfly 0.0033106280025094748


In [3]:
import cv2

cap = cv2.VideoCapture(0)

cont = 0
# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

def classify(frame):
    input_tensor = preprocess(Image.fromarray(frame))
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.

    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    top5_prob, top5_catid = torch.topk(probabilities, 5)

    return categories[top5_catid[0]], top5_prob[0].item()



while True:
    ret, frame = cap.read()     #ret is a flag that indicates if the frame has been captured
    frame = cv2.resize(frame, None, fx=1., fy=1., interpolation=cv2.INTER_AREA)

    cv2.putText(frame, "OpenCV - press ESC to exit", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    top, prob = classify(frame)

    cv2.putText(frame, top, (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    cv2.putText(frame, str(prob), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)


    cv2.imshow('Input', frame)

    c = cv2.waitKey(1)
    if c == 27:
        break       #27 is ascii for ESC

cap.release()
cv2.destroyAllWindows()


Qt: Session management error: Could not open network socket
